In [2]:
import pandas as pd

# 파일 경로 정의
file1 = '[0131최종]상위40퍼센트_구간정보_1번째.xlsx'
file2 = '[0131최종]상위40퍼센트_구간정보_2번째.xlsx'
file3 = '[0131최종]상위40퍼센트_구간정보_3번째.xlsx'

# 각 파일을 DataFrame으로 불러오기
df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)
df3 = pd.read_excel(file3)

# 관련 열 선택 (transfer_info 제외)
columns = ['start_station', 'end_station', 'lines', 'cumulative_passenger', 'climate_pass']
df1 = df1[columns].copy()
df2 = df2[columns].copy()
df3 = df3[columns].copy()

# 'cumulative_passenger' 열 이름 변경 (출처 식별을 위해)
df1.rename(columns={'cumulative_passenger': 'cumulative_passenger_1'}, inplace=True)
df2.rename(columns={'cumulative_passenger': 'cumulative_passenger_2'}, inplace=True)
df3.rename(columns={'cumulative_passenger': 'cumulative_passenger_3'}, inplace=True)

# DataFrame 병합 (외부 조인)
merged_df = pd.merge(df1, df2, on=['start_station', 'end_station', 'lines', 'climate_pass'], how='outer')
merged_df = pd.merge(merged_df, df3, on=['start_station', 'end_station', 'lines', 'climate_pass'], how='outer')

# 누락된 값을 0으로 채우기
merged_df[['cumulative_passenger_1', 'cumulative_passenger_2', 'cumulative_passenger_3']] = merged_df[
    ['cumulative_passenger_1', 'cumulative_passenger_2', 'cumulative_passenger_3']
].fillna(0)

# 가중평균 계산
merged_df['weighted_average'] = (5 * merged_df['cumulative_passenger_1'] +
                                 2 * merged_df['cumulative_passenger_2'] +
                                 2 * merged_df['cumulative_passenger_3']) / 9

# 최종 열 선택 (transfer_info 제외)
final_df = merged_df[['start_station', 'end_station', 'lines', 'weighted_average', 'climate_pass']]

# 고유 조합 개수 계산
unique_combinations = pd.concat([df1, df2, df3]).drop_duplicates(subset=['start_station', 'end_station', 'lines', 'climate_pass'])
num_unique = unique_combinations.shape[0]
num_final = final_df.shape[0]

print(f'모든 파일의 고유 조합 개수: {num_unique}')
print(f'최종 파일의 행 수: {num_final}')

if num_unique == num_final:
    print('검증 완료: 고유 조합의 개수와 최종 파일의 행 수가 일치합니다.')
else:
    print('검증 실패: 고유 조합의 개수와 최종 파일의 행 수가 일치하지 않습니다.')

# 최종 데이터프레임을 새로운 엑셀 파일로 저장
final_df.to_excel('[0131최종]상위40%_3개_데이터_가중평균.xlsx', index=False)

모든 파일의 고유 조합 개수: 307
최종 파일의 행 수: 307
검증 완료: 고유 조합의 개수와 최종 파일의 행 수가 일치합니다.
